# **Операции над полями Галуа GF(2^8) в AES.**

In [1587]:
from pyexpat.errors import messages

primitive = 0x11B # неприводимый многочлен x^8 + x^4 + x^3 + x + 1
check_high_bit = 0x80 # 10000000

In [1588]:
def multiply_GF256(a, b):
    """
    Nhân hai phần tử trong trường GF(2^8) sử dụng đa thức bất khả quy 0x11B.
    """
    result = 0
    for i in range(8):
        if b & 1:  # Nếu bit cuối cùng của b là 1
            result ^= a  # Cộng (XOR) a vào kết quả
        high_bit_set = a & check_high_bit  # Kiểm tra bit cao nhất của a
        a <<= 1
        if high_bit_set:
            a ^= primitive  # Chia lấy dư với đa thức bất khả quy nếu cần
        b >>= 1  # Dịch phải b
    return result

In [1589]:
multiply_GF256(2, 148)

51

In [1590]:
def inverse_GF256(x):
    """Tìm nghịch đảo của x trong GF(2^8) bằng phương pháp lũy thừa nhanh."""
    if x == 0:
        return 0
    y = 1
    for _ in range(254):  # x^(254) mod GF(2^8) sẽ cho ra nghịch đảo
        y = multiply_GF256(y, x)
    return y

In [1591]:
def exp_GF256(elem, exp):
    if exp == 0:
        return 1
    r = exp % 2
    q = exp // 2
    if r:
        return multiply_GF256(elem, multiply_GF256(exp_GF256(elem, q), exp_GF256(elem, q)))
    else:
        return multiply_GF256(exp_GF256(elem, q), exp_GF256(elem, q))

In [1592]:
exp_GF256(2, 8)

27

# **Алгоритм создания S-блока в AES.**

In [1593]:
def affine_transform(byte):
    """Biến đổi Affine của một byte theo công thức AES."""
    mask = 0x63  # Hằng số cố định
    transformed = 0
    for i in range(8):
        # Xác định giá trị bit mới bằng cách XOR các bit theo công thức AES
        new_bit = ((byte >> i) & 1) ^ ((byte >> ((i + 4) % 8)) & 1) ^ \
                  ((byte >> ((i + 5) % 8)) & 1) ^ ((byte >> ((i + 6) % 8)) & 1) ^ \
                  ((byte >> ((i + 7) % 8)) & 1) ^ ((mask >> i) & 1)
        transformed |= (new_bit << i)  # Gán bit vào vị trí tương ứng
    return transformed

In [1594]:
def generate_S_BOX():
    """Tạo bảng S-Box cho AES."""
    S_BOX = [0] * 256
    for i in range(256):
        S_BOX[i] = affine_transform(inverse_GF256(i))
    return S_BOX

In [1595]:
S_BOX = generate_S_BOX()
print("Table S-Box (AES):")
for i in range(16):
    print(" ".join(f"{S_BOX[i * 16 + j]:02X}" for j in range(16)))

Table S-Box (AES):
63 7C 77 7B F2 6B 6F C5 30 01 67 2B FE D7 AB 76
CA 82 C9 7D FA 59 47 F0 AD D4 A2 AF 9C A4 72 C0
B7 FD 93 26 36 3F F7 CC 34 A5 E5 F1 71 D8 31 15
04 C7 23 C3 18 96 05 9A 07 12 80 E2 EB 27 B2 75
09 83 2C 1A 1B 6E 5A A0 52 3B D6 B3 29 E3 2F 84
53 D1 00 ED 20 FC B1 5B 6A CB BE 39 4A 4C 58 CF
D0 EF AA FB 43 4D 33 85 45 F9 02 7F 50 3C 9F A8
51 A3 40 8F 92 9D 38 F5 BC B6 DA 21 10 FF F3 D2
CD 0C 13 EC 5F 97 44 17 C4 A7 7E 3D 64 5D 19 73
60 81 4F DC 22 2A 90 88 46 EE B8 14 DE 5E 0B DB
E0 32 3A 0A 49 06 24 5C C2 D3 AC 62 91 95 E4 79
E7 C8 37 6D 8D D5 4E A9 6C 56 F4 EA 65 7A AE 08
BA 78 25 2E 1C A6 B4 C6 E8 DD 74 1F 4B BD 8B 8A
70 3E B5 66 48 03 F6 0E 61 35 57 B9 86 C1 1D 9E
E1 F8 98 11 69 D9 8E 94 9B 1E 87 E9 CE 55 28 DF
8C A1 89 0D BF E6 42 68 41 99 2D 0F B0 54 BB 16


# **Зашифрование в AES.**

In [1596]:
from bitarray import bitarray
from bitarray.util import ba2int, int2ba

In [1597]:
# key hex_str 128 bit
key = "2B 7E 15 16 28 AE D2 A6 AB F7 15 88 09 CF 4F 3C"
# plaintext hex_str 128 bit: 3243f6a8885a308d313198a2e0370734
plaintext = "32 43 f6 a8 88 5a 30 8d 31 31 98 a2 e0 37 07 34"
S_BOX = generate_S_BOX()
C = [[2, 3, 1, 1],[1, 2, 3, 1],[1, 1, 2, 3],[3, 1, 1, 2]]
versions = {
    128: (4, 10),
    192: (6, 12),
    256: (8, 14)
}

## **Алгоритм расширения ключа**

In [1598]:
def hex_to_bin(hex_string, version=32):
    hex_string = hex_string.replace(" ", "")
    return bitarray(bin(int(hex_string, 16))[2:].zfill(version))

In [1599]:
hex_to_bin('2b')

bitarray('00000000000000000000000000101011')

In [1600]:
def bin_to_hex(bin_array, version=32):
    """Chuyển đổi dữ liệu nhị phân thành chuỗi thập lục phân với định dạng mong muốn."""
    hex_string = format(ba2int(bin_array), 'X').zfill(version // 4)
    return ' '.join(hex_string[i:i+2] for i in range(0, len(hex_string), 2))

In [1601]:
bin_to_hex(bitarray('101011'))

'00 00 00 2B'

In [1602]:
bin_to_hex(bitarray('11011000'))

'00 00 00 D8'

In [1603]:
def RotWord(word: bitarray):
    return word[8:] + word[:8]

In [1604]:
word = '2b 7e 15 16'
print(hex_to_bin(word))
bin_to_hex(RotWord(hex_to_bin(word)))

bitarray('00101011011111100001010100010110')


'7E 15 16 2B'

In [1605]:
def SubWord(word: bitarray):
    new_word = [S_BOX[ba2int(word[i:i+8])] for i in range(0, len(word), 8)]
    return bitarray(''.join(format(i, '08b') for i in new_word))

In [1606]:
word = '2b 7e 15 16'
bin_to_hex(SubWord(hex_to_bin(word)))

'F1 F3 59 47'

In [1607]:
def generate_Rcon(round_id):
    Rcon = exp_GF256(2, round_id - 1)
    Rcon_2b = bitarray(bin(Rcon)[2:].zfill(8)) + bitarray('0'*24)
    return Rcon_2b

In [1608]:
for i in range(1, 11):
    print(f'Rcon[{i}]: {bin_to_hex(generate_Rcon(i))}')

Rcon[1]: 01 00 00 00
Rcon[2]: 02 00 00 00
Rcon[3]: 04 00 00 00
Rcon[4]: 08 00 00 00
Rcon[5]: 10 00 00 00
Rcon[6]: 20 00 00 00
Rcon[7]: 40 00 00 00
Rcon[8]: 80 00 00 00
Rcon[9]: 1B 00 00 00
Rcon[10]: 36 00 00 00


In [1609]:
def expand_key(key, version):
    N = versions[version][0]
    R = versions[version][1] + 1
    
    key_bin = hex_to_bin(key, version=128) 
    
    W = [key_bin[i:i+32] for i in range(0, len(key_bin), 32)]
    for i in range(N, 4 * R):
        temp = W[i - 1]
        if i % N == 0:
            temp = SubWord(RotWord(temp)) ^ generate_Rcon(i // N)
        elif version == 256 and i % N == 4:
            temp = SubWord(temp)
        W.append(W[i - N] ^ temp)
    
    # chia thành các key vòng tương ứng cho các vòng AES
    return [W[i:i+4] for i in range(0, len(W), 4)]

In [1610]:
# keys = expand_key(key, 128)
# for key_ in keys:
#     print(key_)  

## **Раунд AES**

In [1611]:
def SubBytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = S_BOX[state[i][j]]
    return state

In [1612]:
def ShiftRows(state):
    for i in range(4):
        state[i] = state[i][i:] + state[i][:i]
    return state

In [1613]:
def MixColumns(state):
    new_state = [[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            new_state[j][i] = (multiply_GF256(C[i][0], state[0][j]) ^ \
                                multiply_GF256(C[i][1], state[1][j]) ^ \
                                multiply_GF256(C[i][2], state[2][j]) ^ \
                               multiply_GF256(C[i][3], state[3][j]))
            
    return new_state

In [1614]:
state = [
    [0xD4, 0xE0, 0xB8, 0x1E],
    [0xBF, 0xB4, 0x41, 0x27],
    [0x5D, 0x52, 0x11, 0x98],
    [0x30, 0xAE, 0xF1, 0xE5]
]
new_state = MixColumns(state)
# in new_state dạng hex
for i in range(4):
    for j in range(4):
        print(hex(new_state[i][j]), end=' ')
    print()

0x4 0x66 0x81 0xe5 
0xe0 0xcb 0x19 0x9a 
0x48 0xf8 0xd3 0x7a 
0x28 0x6 0x26 0x4c 


In [1615]:
def AddRoundKey(state, key):
    key_mat = [[0]*4 for _ in range(4)]
    print('key:')
    for i in range(4):
        for j in range(4):
            key_mat[i][j] = ba2int(key[j][i*8:(i+1)*8])
            print(f"{key_mat[i][j]:02X}", end=' ')
        print()
    
    new_state = [[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            new_state[i][j] = state[i][j] ^ key_mat[i][j]

    return new_state

In [1616]:
def printState(state, name="state"):
    print(name)
    for i in range(4):
        for j in range(4):
            print(f"{state[i][j]:02X}", end=' ')
        print()

In [1617]:
def trans_mat(state):
    new_state = [[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            new_state[j][i] = state[i][j]
    return new_state

In [1618]:
def Round_AES(state, key):
    state = SubBytes(state)
    printState(state, "SubBytes")
    state = ShiftRows(state)
    printState(state, "ShiftRows")
    state = MixColumns(state)
    printState(state, "MixColumns")
    state = AddRoundKey(trans_mat(state), key)
    printState(state, "AddRoundKey")
    return state

## **Function encrypt AES**

In [1619]:
from copy import deepcopy

In [1620]:
def encrypt_AES(plaintext, key, version):
    N = versions[version][0]
    R = versions[version][1]
    
    plaintext = hex_to_bin(plaintext, version=128)
    
    keys_round = expand_key(key, version)
    key_states = []
    
    state = [[0]*4 for _ in range(4)]
    print("Start state:")
    for i in range(4):
        for j in range(4):
            state[i][j] = ba2int(plaintext[j*32 + i*8:j*32 + (i+1)*8])
            print(f'{state[i][j]:02X}', end=' ')
        print()
   
    state = AddRoundKey(state, keys_round[0])
    print("Round 0:")
    printState(state)
    key_states.append((deepcopy(keys_round[0]), deepcopy(state)))
    
    for i in range(1, R):
        print(f"Round {i}:")
        state = Round_AES(state, keys_round[i])
        key_states.append((deepcopy(keys_round[i]), deepcopy(state)))
    
    print(f"Round {R}:")
    state = SubBytes(state)
    printState(state, "SubBytes")
    state = ShiftRows(state)
    printState(state, "ShiftRows")
    state = AddRoundKey(state, keys_round[R])
    printState(state, "AddRoundKey")
    key_states.append((deepcopy(keys_round[R]), deepcopy(state)))
    return state, key_states

In [1621]:
def arr_to_hex_str(int_array):
    if type(int_array[0]) is bitarray:
        # chuyển thành int
        int_array = [ba2int(elem) for elem in int_array]
    return ' '.join(f"{i:02X}" for i in int_array)

In [1622]:
def convert_state_to_hex_str(state):
    return ' '.join([arr_to_hex_str(col) for col in zip(*state)])

In [1623]:
cipher_state, _ = encrypt_AES(plaintext, key, 128)
# chuyển thành dạng hex str
print("-----------------")
for i in range(4):
    for j in range(4):
        print(f"{cipher_state[i][j]:02X}", end=' ')
    print()
  
ciphertext = convert_state_to_hex_str(cipher_state)
print(ciphertext)

Start state:
32 88 31 E0 
43 5A 31 37 
F6 30 98 07 
A8 8D A2 34 
key:
2B 28 AB 09 
7E AE F7 CF 
15 D2 15 4F 
16 A6 88 3C 
Round 0:
state
19 A0 9A E9 
3D F4 C6 F8 
E3 E2 8D 48 
BE 2B 2A 08 
Round 1:
SubBytes
D4 E0 B8 1E 
27 BF B4 41 
11 98 5D 52 
AE F1 E5 30 
ShiftRows
D4 E0 B8 1E 
BF B4 41 27 
5D 52 11 98 
30 AE F1 E5 
MixColumns
04 66 81 E5 
E0 CB 19 9A 
48 F8 D3 7A 
28 06 26 4C 
key:
A0 88 23 2A 
FA 54 A3 6C 
FE 2C 39 76 
17 B1 39 05 
AddRoundKey
A4 68 6B 02 
9C 9F 5B 6A 
7F 35 EA 50 
F2 2B 43 49 
Round 2:
SubBytes
49 45 7F 77 
DE DB 39 02 
D2 96 87 53 
89 F1 1A 3B 
ShiftRows
49 45 7F 77 
DB 39 02 DE 
87 53 D2 96 
3B 89 F1 1A 
MixColumns
58 4D CA F1 
1B 4B 5A AC 
DB E7 CA A8 
1B 6B B0 E5 
key:
F2 7A 59 73 
C2 96 35 59 
95 B9 80 F6 
F2 43 7A 7F 
AddRoundKey
AA 61 82 68 
8F DD D2 32 
5F E3 4A 46 
03 EF D2 9A 
Round 3:
SubBytes
AC EF 13 45 
73 C1 B5 23 
CF 11 D6 5A 
7B DF B5 B8 
ShiftRows
AC EF 13 45 
C1 B5 23 73 
D6 5A CF 11 
B8 7B DF B5 
MixColumns
75 EC 09 93 
20 0B 63 33 
53 C0 CF 7

# **Расшифрование в AES.**

In [1624]:
def generate_inv_S_BOX(S_BOX):
    """Tạo bảng S-Box nghịch đảo cho AES."""
    INV_S_BOX = [0] * 256
    for i in range(256):
        INV_S_BOX[S_BOX[i]] = i
    return INV_S_BOX

In [1625]:
INV_S_BOX = generate_inv_S_BOX(S_BOX)
inv_C = [[14, 11, 13, 9],[9, 14, 11, 13],[13, 9, 14, 11],[11, 13, 9, 14]]

# **Раунд AES**

In [1626]:
def InvSubBytes(state):
    for i in range(4):
        for j in range(4):
            state[i][j] = INV_S_BOX[state[i][j]]
    return state

In [1627]:
def InvShiftRows(state):
    for i in range(4):
        state[i] = state[i][-i:] + state[i][:-i]
    return state

In [1628]:
state = [['A2', '8A', '31', '43'],
        ['F2', '2D', 'A0', 'A4'],
        ['C4', 'C8', '8C', '10'],
        ['D4', 'C7', 'E3', '48']]
       
InvShiftRows(state)

[['A2', '8A', '31', '43'],
 ['A4', 'F2', '2D', 'A0'],
 ['8C', '10', 'C4', 'C8'],
 ['C7', 'E3', '48', 'D4']]

In [1629]:
def InvMixColumns(state):
    new_state = [[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            new_state[j][i] = (multiply_GF256(inv_C[i][0], state[0][j]) ^ \
                                multiply_GF256(inv_C[i][1], state[1][j]) ^ \
                                multiply_GF256(inv_C[i][2], state[2][j]) ^ \
                               multiply_GF256(inv_C[i][3], state[3][j]))
            
    return new_state

In [1630]:
def AddRoundKey(state, key):
    key_mat = [[0]*4 for _ in range(4)]
    # print('key:')
    for i in range(4):
        for j in range(4):
            key_mat[i][j] = ba2int(key[j][i*8:(i+1)*8])
        #     print(f"{key_mat[i][j]:02X}", end=' ')
        # print()
    
    new_state = [[0]*4 for _ in range(4)]
    for i in range(4):
        for j in range(4):
            new_state[i][j] = state[i][j] ^ key_mat[i][j]

    return new_state

In [1631]:
def Round_AES_inv(state, key):
    state = InvShiftRows(state)
    #printState(state, "InvShiftRows")
    state = InvSubBytes(state)
    #printState(state, "InvSubBytes")
    state = AddRoundKey(state, key)
    #printState(state, "AddRoundKey")
    state = InvMixColumns(state)
    state = trans_mat(state)
    #printState(state, "InvMixColumns")
    return state

## **Function decrypt AES**

In [1632]:
def decrypt_AES(ciphertext, key, version):
    N = versions[version][0]
    R = versions[version][1]
    
    ciphertext = hex_to_bin(ciphertext, version=128)
    
    keys_round = expand_key(key, version)[::-1]
    states = []
    
    state = [[0]*4 for _ in range(4)]
    # print("Start state:")
    for i in range(4):
        for j in range(4):
            state[i][j] = ba2int(ciphertext[j*32 + i*8:j*32 + (i+1)*8])
        #     print(f'{state[i][j]:02X}', end=' ')
        # print()
    states.append(deepcopy(state))
    
    state = AddRoundKey(state, keys_round[0])
    # print("Round 0:")
    # printState(state)
    states.append(deepcopy(state))
    
    for i in range(1, R):
        # print(f"Round {i}:")
        state = Round_AES_inv(state, keys_round[i])
        states.append(deepcopy(state))
        
    state = InvShiftRows(state)
    state = InvSubBytes(state)
    state = AddRoundKey(state, keys_round[R])
    states.append(deepcopy(state))
    # print(f"Round {R}:")
    # printState(state)
    
    return state

In [1633]:
print(ciphertext)
plaintext_state = decrypt_AES(ciphertext, key, 128)
# chuyển thành dạng hex str
print("-----------------")
for i in range(4):
    for j in range(4):
        print(f"{plaintext_state[i][j]:02X}", end=' ')
    print()

39 25 84 1D 02 DC 09 FB DC 11 85 97 19 6A 0B 32
-----------------
32 88 31 E0 
43 5A 31 37 
F6 30 98 07 
A8 8D A2 34 


In [1634]:
plaintext = convert_state_to_hex_str(plaintext_state)
print(plaintext)

32 43 F6 A8 88 5A 30 8D 31 31 98 A2 E0 37 07 34


## **AES version 256**

In [1635]:
key = "FD E8 F7 A9 B8 6C 3B FF 07 C0 D3 9D 04 60 5E DD 14 A3 D4 B6 33 45 4D 7C 5B 21 3A 5B 9A 0F 58 6C"
plaintext = "32 43 F6 A8 88 5A 30 8D 31 31 98 A2 E0 37 07 34"

In [1636]:
cipher_state, key_states = encrypt_AES(plaintext, key, 256)

Start state:
32 88 31 E0 
43 5A 31 37 
F6 30 98 07 
A8 8D A2 34 
Round 0:
state
CF 30 36 E4 
AB 36 F1 57 
01 0B 4B 59 
01 72 3F E9 
Round 1:
SubBytes
8A 04 05 69 
62 05 A1 5B 
7C 2B B3 CB 
7C 40 75 1E 
ShiftRows
8A 04 05 69 
05 A1 5B 62 
B3 CB 7C 2B 
1E 7C 40 75 
MixColumns
AD 50 D0 0F 
47 67 AC 9E 
DB 77 66 A8 
2A A5 C2 18 
AddRoundKey
B9 74 80 B0 
F3 22 56 AA 
04 E1 5C 9A 
B9 E2 F3 74 
Round 2:
SubBytes
56 92 CD E7 
0D 93 B1 AC 
F2 F8 4A B8 
56 98 0D 92 
ShiftRows
56 92 CD E7 
93 B1 AC 0D 
4A B8 F2 F8 
92 56 98 0D 
MixColumns
DA 27 FC 1C 
19 6E B2 08 
04 1B 2D 39 
37 E3 16 DD 
AddRoundKey
50 2B 31 06 
A5 80 35 AD 
5B 2E 62 07 
0D E6 4A 73 
Round 3:
SubBytes
53 F1 C7 6F 
06 CD 96 95 
39 31 AA C5 
D7 8E D6 8F 
ShiftRows
53 F1 C7 6F 
CD 96 95 06 
AA C5 39 31 
8F D7 8E D6 
MixColumns
CF B8 5B 97 
4A 45 94 EE 
86 33 A9 F9 
33 E6 6A 31 
AddRoundKey
1C AA 3D 12 
34 8C DB 01 
0D 8F 88 13 
C5 C0 8C 28 
Round 4:
SubBytes
9C AC 27 C9 
18 64 B9 7C 
D7 73 C4 7D 
A6 BA 64 34 
ShiftRows
9C AC 27 C9

In [1637]:
def arr_key_to_hex_str(key_array):
    return ' '.join(bin_to_hex(key) for key in key_array if type(key) is bitarray)

In [1638]:
key_states_hex_str = [(arr_key_to_hex_str(key), convert_state_to_hex_str(state))  
                        for key, state in key_states]
i = -1
for key_hex, state_hex in key_states_hex_str:
    print(f"Key {i}:", key_hex)
    print(f"State {i}:", state_hex)
    i += 1

Key -1: FD E8 F7 A9 B8 6C 3B FF 07 C0 D3 9D 04 60 5E DD
State -1: CF AB 01 01 30 36 0B 72 36 F1 4B 3F E4 57 59 E9
Key 0: 14 A3 D4 B6 33 45 4D 7C 5B 21 3A 5B 9A 0F 58 6C
State 0: B9 F3 04 B9 74 22 E1 E2 80 56 5C F3 B0 AA 9A 74
Key 1: 8A 82 A7 11 32 EE 9C EE 35 2E 4F 73 31 4E 11 AE
State 1: 50 A5 5B 0D 2B 80 2E E6 31 35 62 4A 06 AD 07 73
Key 2: D3 8C 56 52 E0 C9 1B 2E BB E8 21 75 21 E7 79 19
State 2: 1C 34 0D C5 AA 8C 8F C0 3D DB 88 8C 12 01 13 28
Key 3: 1C 34 73 EC 2E DA EF 02 1B F4 A0 71 2A BA B1 DF
State 3: 63 03 44 9B 66 3E F1 7E BC F3 9B DC 9C B2 2A 3C
Key 4: 36 78 9E CC D6 B1 85 E2 6D 59 A4 97 4C BE DD 8E
State 4: E9 2B D9 B1 56 B8 56 77 16 8C CE E9 41 E8 B0 3A
Key 5: B6 F5 6A C5 98 2F 85 C7 83 DB 25 B6 A9 61 94 69
State 5: 35 35 8E 1B 62 AC C6 07 7B 1B 97 20 13 CD BD 8B
Key 6: E5 97 BC 35 33 26 39 D7 5E 7F 9D 40 12 C1 40 CE
State 6: CF 86 F7 F7 43 E8 22 A2 1C D1 67 16 4A FB 1A 1E
Key 7: C6 FC E1 0C 5E D3 64 CB DD 08 41 7D 74 69 D5 14
State 7: 88 BD 77 73 E2 20 A7 A9 BD 08 4C D1 DB

In [1639]:
# in ra file excel
import pandas as pd

df = pd.DataFrame(key_states_hex_str, columns=['Key', 'State'])
df.to_excel('output/AES_256.xlsx', index=True)

# **Mode CBC AES**

In [1640]:
IV = "00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F"
key = "FD E8 F7 A9 B8 6C 3B FF 07 C0 D3 9D 04 60 5E DD" #14 A3 D4 B6 33 45 4D 7C 5B 21 3A 5B 9A 0F 58 6C"
message = "The quick brown fox jumps over the lazy dog."

In [1641]:
"""Viết hàm chuyển chuỗi thành dạng nhị phân dưới dạng các block 64 bit"""
def str_to_block_128_bit(message: str, mode_padding="zeros"):
    binary_string = bitarray()
    binary_string.frombytes(message.encode("ISO-8859-1"))
    padding = 128 - len(binary_string) % 128
    if mode_padding == "zeros":
        binary_string += bitarray(padding * "0")
    elif mode_padding == "one-zeros":
        binary_string += bitarray("1")
        binary_string += bitarray("0" * (padding - 1))
    elif mode_padding == "PKCS7" and padding % 8 == 0:
        length = padding // 8
        binary_string += bitarray(bin(length)[2:].zfill(8) * length)
    return [binary_string[i:i+128] for i in range(0, len(binary_string), 128)]

In [1642]:
blocks = str_to_block_128_bit(message, "PKCS7")

for block in blocks:
    hex_block = bin_to_hex(block, version=128)
    print(hex_block)

54 68 65 20 71 75 69 63 6B 20 62 72 6F 77 6E 20
66 6F 78 20 6A 75 6D 70 73 20 6F 76 65 72 20 74
68 65 20 6C 61 7A 79 20 64 6F 67 2E 04 04 04 04


In [1643]:
def block_128_bit_to_str(blocks: bitarray) -> str:
    binary_string = bitarray()
    for block in blocks:
        binary_string += block
    return binary_string.tobytes().decode("ISO-8859-1")

In [1644]:
def encrypt_AES_mode_CBC(message: str, key: bitarray, IV: bitarray, version=128):
    blocks = str_to_block_128_bit(message, mode_padding="PKCS7")
    
    IV = hex_to_bin(IV, version=128)
    
    ciphertexts = []
    
    for block in blocks:
        block = block ^ IV
        IV_state, _ = encrypt_AES(bin_to_hex(block), key, version)
        IV = hex_to_bin(convert_state_to_hex_str(IV_state), version=128)
        ciphertexts.append(IV)
    
    ciphertext = ' '.join([bin_to_hex(block, version=128) for block in ciphertexts])
        
    return ciphertext

In [1645]:
ciphertext_hex = encrypt_AES_mode_CBC(message, key, IV, 128)
print("ciphertext:", ciphertext_hex)

Start state:
54 75 63 63 
69 70 29 7A 
67 6F 68 60 
23 64 79 2F 
Round 0:
state
A9 CD 64 67 
81 1C E9 1A 
90 54 BB 3E 
8A 9B E4 F2 
Round 1:
SubBytes
D3 BD 43 85 
0C 9C 1E A2 
60 20 EA B2 
7E 14 69 89 
ShiftRows
D3 BD 43 85 
9C 1E A2 0C 
EA B2 60 20 
89 7E 14 69 
MixColumns
61 5C 00 11 
8F 32 5E 8C 
0F A8 1D 2F 
4C 94 72 6A 
AddRoundKey
4D 1B 9C DB 
EC EE B4 E8 
36 53 C3 F2 
4A 28 16 8E 
Round 2:
SubBytes
E3 AF DE B9 
CE 28 8D 9B 
05 ED 2E 89 
D6 34 47 19 
ShiftRows
E3 AF DE B9 
28 8D 9B CE 
2E 89 05 ED 
19 D6 34 47 
MixColumns
92 D8 BC 0A 
96 F8 4A 59 
20 C8 13 8F 
8A 55 7F 7D 
AddRoundKey
AC 3C 19 24 
A5 59 75 94 
E3 18 9F 73 
D9 2E C1 D7 
Round 3:
SubBytes
91 EB D4 36 
06 CB 9D 22 
11 AD DB 8F 
35 31 78 0E 
ShiftRows
91 EB D4 36 
CB 9D 22 06 
DB 8F 11 AD 
0E 35 31 78 
MixColumns
AA 64 E5 A4 
CB 75 2C 5E 
F5 92 87 36 
B3 AE F9 01 
AddRoundKey
E8 23 24 CC 
E7 57 0D F0 
16 8D AA D8 
93 1E 38 A5 
Round 4:
SubBytes
9B 26 36 4B 
94 5B D7 8C 
47 5D AC 61 
DC 72 07 06 
ShiftRows
9B 26 36 4B

# **Request server and check valid padding**

In [1646]:
from typing import Union

In [1647]:
PADDING_INVALID = False
PADDING_VALID = True

In [1648]:
def check_valid_padding(plaintext: Union[bitarray, str]) -> bool:
    if isinstance(plaintext, str):
        plaintext = hex_to_bin(plaintext, version=128)
    plaintext_hex = bin_to_hex(plaintext, version=128).split(" ")
    length_padding = ba2int(plaintext[-8:])
    
    n = len(plaintext_hex)
    for i in range(1, length_padding + 1):
        if plaintext_hex[n-i] != plaintext_hex[-1]:
            return False
    return True

In [1649]:
plaintext_check = "68 65 20 6C 61 7A 79 20 64 6F 67 2E 04 04 04 04"
check_valid_padding(plaintext_check)

True

In [1650]:
def response_server(plaintext: Union[bitarray, str]) -> bool:
    if check_valid_padding(plaintext):
        return PADDING_VALID
    else:
        return PADDING_INVALID

In [1651]:
def request_server(C1: bitarray, C2: bitarray):
    I2_decrypted = decrypt_AES(bin_to_hex(C2, version=128), key, 128)
    I2 = hex_to_bin(convert_state_to_hex_str(I2_decrypted), version=128)
    P2 = I2 ^ C1
    #print("P2: ", bin_to_hex(P2, version=128))
    return response_server(P2)

# **Padding Oracle Attack**

In [1652]:
message = "My name is Duy"

In [1653]:
C1 = "0A 44 96 65 37 63 9E 22 BB 5A 36 C1 70 56 9A 56"
C2 = "6C 37 BC 60 0D 45 E9 B9 DD 66 FD E0 2A 2D C2 60"

In [1654]:
blocks = str_to_block_128_bit(message, "PKCS7")

for block in blocks:
    hex_block = bin_to_hex(block, version=128)
    print(hex_block)

4D 79 20 6E 61 6D 65 20 69 73 20 44 75 79 02 02


In [1655]:
def change_byte(C: Union[bitarray, str], index: int, value: Union[hex, int]) -> bitarray:
    value = format(value, "08b")    
    C1 = deepcopy(C)
    if isinstance(C, str):
        C1 = hex_to_bin(C1, version=128)
    
    C1 = C1[:index*8] + bitarray(value) + C1[(index+1)*8:]
    return C1

In [1656]:
def get_byte(C: Union[bitarray, str], index: int) -> int:
    C1 = deepcopy(C)
    if isinstance(C1, str):
        C1 = hex_to_bin(C1, version=128)
    return ba2int(C1[index*8:(index+1)*8])

In [1657]:
get_byte(C1, 15)

86

In [1658]:
bin_to_hex(change_byte(C1, 0, 0x0C), version=128)

'0C 44 96 65 37 63 9E 22 BB 5A 36 C1 70 56 9A 56'

In [1659]:
def get_length_padding(C1: Union[bitarray, str], C2: Union[bitarray, str]) -> int:
    if isinstance(C1, str):
        C1 = hex_to_bin(C1, version=128)
    if isinstance(C2, str):
        C2 = hex_to_bin(C2, version=128)
    for i in range(16):
        byte = get_byte(C1, i)
        byte += 1
        R1 = change_byte(C1, i, byte)
        if request_server(R1, C2) == PADDING_INVALID:
            return 16 - i
    
    return 0

In [1660]:
get_length_padding(C1, C2)

2

In [1672]:
def remove_padding_from_block(block: Union[bitarray, str], length_padding: int) -> bitarray:
    if isinstance(block, str):
        block = hex_to_bin(block, version=128)
    return block[:-length_padding*8]

In [1673]:
def padding_oracle_attack(C1: Union[bitarray, str], C2: Union[bitarray, str]):
    if isinstance(C1, str):
        C1 = hex_to_bin(C1, version=128)
    if isinstance(C2, str):
        C2 = hex_to_bin(C2, version=128)
    real_length_padding = get_length_padding(C1, C2)
    
    I2 = []
    for i in range(real_length_padding):
        byte = get_byte(C1, 16 - i - 1)
        I2.append(byte ^ real_length_padding)
    
    length_padding = real_length_padding
    while len(I2) < 16:
        R1 = deepcopy(C1)
        for i in range(length_padding):
            # R1 = I2 xor length_padding
            byte = I2[i] ^ (length_padding + 1)
            print(f"{hex(I2[i])} + {hex(length_padding + 1)} = {hex(byte)}")
            R1 = change_byte(R1, 16 - i - 1, byte)
        
        # lặp byte C1[length_padding+1] (đếm từ phải qua) cho đến khi padding hợp lệ
        for i in range(256):
            
            R1 = change_byte(R1, 16 - length_padding - 1, i)
            if request_server(R1, C2) == PADDING_VALID:
                I2.append(i ^ (length_padding + 1))
                print('Padding valid in R1: ', bin_to_hex(R1, version=128))
                break
        length_padding += 1
    
    I2 = I2[::-1]
    I2 = bitarray(''.join(format(i, '08b') for i in I2))
    P2 = I2 ^ C1
    P2 = remove_padding_from_block(P2, real_length_padding)
    return P2

In [1674]:
result = padding_oracle_attack(C1, C2)

0x54 + 0x3 = 0x57
0x98 + 0x3 = 0x9b
Padding valid in R1:  0A 44 96 65 37 63 9E 22 BB 5A 36 C1 70 2C 9B 57
0x54 + 0x4 = 0x50
0x98 + 0x4 = 0x9c
0x2f + 0x4 = 0x2b
Padding valid in R1:  0A 44 96 65 37 63 9E 22 BB 5A 36 C1 01 2B 9C 50
0x54 + 0x5 = 0x51
0x98 + 0x5 = 0x9d
0x2f + 0x5 = 0x2a
0x5 + 0x5 = 0x0
Padding valid in R1:  0A 44 96 65 37 63 9E 22 BB 5A 36 80 00 2A 9D 51
0x54 + 0x6 = 0x52
0x98 + 0x6 = 0x9e
0x2f + 0x6 = 0x29
0x5 + 0x6 = 0x3
0x85 + 0x6 = 0x83
Padding valid in R1:  0A 44 96 65 37 63 9E 22 BB 5A 10 83 03 29 9E 52
0x54 + 0x7 = 0x53
0x98 + 0x7 = 0x9f
0x2f + 0x7 = 0x28
0x5 + 0x7 = 0x2
0x85 + 0x7 = 0x82
0x16 + 0x7 = 0x11
Padding valid in R1:  0A 44 96 65 37 63 9E 22 BB 2E 11 82 02 28 9F 53
0x54 + 0x8 = 0x5c
0x98 + 0x8 = 0x90
0x2f + 0x8 = 0x27
0x5 + 0x8 = 0xd
0x85 + 0x8 = 0x8d
0x16 + 0x8 = 0x1e
0x29 + 0x8 = 0x21
Padding valid in R1:  0A 44 96 65 37 63 9E 22 DA 21 1E 8D 0D 27 90 5C
0x54 + 0x9 = 0x5d
0x98 + 0x9 = 0x91
0x2f + 0x9 = 0x26
0x5 + 0x9 = 0xc
0x85 + 0x9 = 0x8c
0x16 + 0x9 = 0

In [1675]:
plaintext = block_128_bit_to_str([result])
print("plaintext:", plaintext)

plaintext: My name is Duy
